In [45]:
from __future__ import annotations
import os
import inspect
from dotenv import load_dotenv
import json, pathlib
from typing import List, Any
from agents import Agent, handoff, RunContextWrapper, Runner, RunHooks, AgentHooks, function_tool,Tool
from pydantic import BaseModel
from dataclasses import dataclass
from pathlib import Path

import logging
logging.basicConfig(level=logging.INFO)

In [17]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")


AGENTS_DB = pathlib.Path("./agent_data_base/agents.json")
AGENTS_DB.parent.mkdir(parents=True, exist_ok=True)

if not api_key:
    raise ValueError("OPENAI API KEY IS NOT FOUND IN THE ENVIORMENT VARIABLES")

## Classic Agent Tools

In [3]:
@dataclass
class MyCtx: 
    user_id: str
    call_count: int = 0

In [4]:
@function_tool
def increment_counter(ctx: RunContextWrapper[MyCtx], note: str) -> str:
    ctx.context.call_count += 1
    return (
        f"User {ctx.context.user_id} 增加计数，当前 = {ctx.context.call_count}"
        f"备注: {note}"
    )

In [5]:
agent = Agent(
    name="Counter Agent",
    instructions="Call the `increment` tool whenever the user asks.",
    tools=[increment_counter],
)

In [ ]:
ctx_obj = MyCtx(user_id="alice")

res1 = await Runner.run(
        agent,
        'Please increment with note "first hit"',
        context=ctx_obj
    )

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [7]:
print(res1.final_output)
print("Ctx after 1st:", ctx_obj)

The counter has been incremented. Note: "first hit".
Ctx after 1st: MyCtx(user_id='alice', call_count=1)


## Auto Create Agent tools

In [32]:
class SelfSpawningAgent(Agent):
    """Any instance can further spawn children via auto_spawn."""
    def __init__(
        self, name: str, instructions: str, base_tools: List[Any] = None,
    ):
        super().__init__(name=name, instructions=instructions, tools=base_tools or [])

In [ ]:
@function_tool
def add_agent(ctx: RunContextWrapper[SelfSpawningAgent], 
                new_agent_name: str, new_agent_instruction: str,) -> None:
    owner = ctx.context
    logging.info(f"🚀 AutoSpawn: {owner.name} → 生成新代理『{new_agent_name}』")
    new_agent = SelfSpawningAgent(name = new_agent_name, 
                    instructions=new_agent_instruction)
    owner.handoffs.append(new_agent)
    return None

In [ ]:
@function_tool
def update(ctx: RunContextWrapper[SelfSpawningAgent], 
              new_agent_name: str, new_agent_instruction: str):
    ctx.context.name = new_agent_name
    ctx.context.name = new_agent_instruction

    return None

In [ ]:
@function_tool
def update(ctx: RunContextWrapper[SelfSpawningAgent], 
              new_agent_name: str, new_agent_instruction: str):
    ctx.context.name = new_agent_name
    ctx.context.name = new_agent_instruction

    return None

In [ ]:
triage_agent = SelfSpawningAgent(
    name="Triage",
    instructions=(
        "You are a helpful agent, call the `add_agent` tool whenever the user asks. with name "
        "temp and instruction None"
    ),
    base_tools=[add_agent],
)

In [35]:
res1 = await Runner.run(
        triage_agent,
        'call tool show',
        context=triage_agent
    )

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:🚀 AutoSpawn: Triage → 生成新代理『temp』
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [36]:
res1.final_output

'I\'ve added a new agent named "temp" with the instruction "None." If you need anything else, let me know!'

In [37]:
triage_agent.handoffs

[SelfSpawningAgent(name='temp', instructions='None', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)]

## Search Agent

In [61]:
@dataclass
class QueryAgentResult:
    name: str
    id: str
    instructions: str

In [ ]:
class SearchHook(RunHooks):
    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        print("🔍 I'm searching ...")
    
    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        print(
            f"### Tool {tool.name} ended with result {result}."
        )


class SelfSpawningAgent(Agent):
    """Any instance can further spawn children via auto_spawn."""
    def __init__(
        self, name: str, instructions: str, base_tools: List[Any] = None,
    ):
        super().__init__(name=name, instructions=instructions, tools=base_tools or [])
        query_agent = Agent(
            name = "query_agent",
            instructions= (
                "You are a search agent and will return the best match agent based on user input"
                f"The current Agent we have is {self.get_all_agents(AGENTS_DB)}"
                ),
            output_type = QueryAgentResult,
        )
        self.tools.append(query_agent.as_tool(tool_name = "query_agent", 
                                              tool_description = "An agent can help you find specialized agent"
                                              ))

    def get_all_agents(self, file_path: str) -> dict:
        file_path = Path("./agent_data_base/agents.json")
        with file_path.open("r", encoding="utf-8") as f:
            data = json.load(f)
        
        return data
        

In [66]:
triage_agent = SelfSpawningAgent(
    name="Triage",
    instructions=(
        "You will search for an agent to help (call your tool)"
    ),
)


In [67]:
res1 = await Runner.run(triage_agent, 
                        "Is two surjective function's composition also composition?",
                            hooks=SearchHook(),
                        )

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


🔍 I'm searching ...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


### Tool query_agent ended with result {"response":{"name":"Math Agent","id":"12344","instructions":"You are a general Math Agent"}}.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [68]:
res1.final_output

'To determine if the composition of two surjective functions is also surjective, we can look into the properties of surjective functions.\n\nA function \\( f: A \\to B \\) is surjective (or onto) if every element in the set \\( B \\) has a pre-image in the set \\( A \\). That is, for every \\( b \\in B \\), there exists at least one \\( a \\in A \\) such that \\( f(a) = b \\).\n\nNow, consider two surjective functions:\n- \\( f: A \\to B \\)\n- \\( g: B \\to C \\)\n\nTheir composition \\( g \\circ f \\) is defined as \\( g(f(a)) \\) for \\( a \\in A \\).\n\nTo check if \\( g \\circ f \\) is surjective:\n- For every \\( c \\in C \\), since \\( g \\) is surjective, there exists a \\( b \\in B \\) such that \\( g(b) = c \\).\n- Since \\( f \\) is surjective, for this \\( b \\), there exists an \\( a \\in A \\) such that \\( f(a) = b \\).\n- Therefore, \\( g(f(a)) = c \\).\n\nThis means there exists an \\( a \\) for every \\( c \\in C \\) such that \\( g(f(a)) = c \\), demonstrating that t